# Titanic Survivor Analysis...

In [2]:
print("Hello Shape AI...")
print("Harshit this side...")

Hello Shape AI...
Harshit this side...


In [3]:
import pandas as pd
import numpy as np

# Reading Data using Pandas...

We are using pandas read_csv function to read the csv file in python and pandas Dataframe method  to convert file into the data frame...

In [5]:
df = pd.DataFrame(pd.read_csv('train.csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.shape

(891, 12)

**Description of the attributes of the dataset**

Pclass: Passenger Class (1 = 1st ;2 = 2nd ;3 = 3rd)

survival: Survival(0 = No; 1 = Yes)

name:Name

sex:Gender

age:Age

sibsp:Number of Siblings/Spouses Aboard

parch:Number of Parents/Children Aboard

ticket:Ticket Number

fare:Passenger Fare(British pound) 

cabin:Cabin

embarked:Port of Embarkation(C = Cherbourg; Q = Queenstown; S=Southampton)

# Handeling Null Values...

The dataset may contain many rows and columns for which some values are missing, we can't leave those missing values as it is.


In [8]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Separating out the columns which have more than 35% of the values missing in the dataset...


In [10]:
drop_col = df.isnull().sum()[df.isnull().sum()>(35/100 * df.shape[0])]
drop_col

Cabin    687
dtype: int64

In [12]:
drop_col.index

Index(['Cabin'], dtype='object')

In [13]:
df.drop(drop_col.index, axis=1, inplace=True)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [14]:
df.fillna(df.mean(), inplace = True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Because **Embarked** contains string values, we can ignore its null values uptill now...

In [16]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For Embarked attribute, we fill the NULL values with the most frequent value in the column...


In [18]:
df['Embarked'].fillna('S', inplace = True)

In [20]:
df.isnull().sum()    #now you'll see that all the values have been filled or removed...

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [21]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


As discussed...

sibsp:Number of Siblings/Spouses Aboard

parch:Number of Parents/Children Aboard

So we will make a new column family_size by combining these two columns...


In [22]:
df['Familysize'] = df['SibSp'] + df['Parch']
df.drop(['SibSp', 'Parch'], axis = 1, inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,Familysize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
Familysize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


FamilySize in the ship does not have much correlance with the survival rate...

Let's check if the person was alone or not can affect the survival rate...

In [30]:
df['Alone'] = [0 if df['Familysize'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Familysize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,373450,8.0500,S,0,1


In [24]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

If the person is alone he/she has chance of surviving...


> Indented block
The reason might be the person who is travelling with his family might be belonging to rich class and might be prioritized over other.


In [26]:
df[['Alone', 'Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


In [27]:
df['Sex'] = [0 if df['Sex'][i]=='male' else 1 for i in df.index]  #1 for female...
df.groupby(['Sex'])['Survived'].mean() 

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

It shows,female passengers have more chance of surviving than male ones.

It shows women were priritized over men.

In [29]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

# Conclusion...



*   Female passengers were prioritized over men.
*   People with high class or rich people have higher survival rate than others.
*   Passengers travelling with their family have higher survival rate.
*   Passengers who boarded the ship at Cherbourg, survived more in proportion than the others...



